In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split as tts

/Users/noelkonagai/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [10]:
data = pd.read_csv('pima-indians-diabetes.csv')

##train-test split
train_size = round(len(data) * 0.7)

X = data[data.columns.difference(['Class'])]
Y = data[['Class']]

x_train = X[:train_size]
y_train = Y[:train_size]

x_test = X[train_size:]
y_test = Y[train_size:]

data[:5]

,Number_pregnant,Glucose_concentration,Blood_pressure,Triceps,Insulin,BMI,Pedigree,Age,Class,Group
0,6,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,50,1,B
1,1,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,31,0,C
2,8,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,32,1,B
3,1,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,21,0,B
4,0,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,33,1,C


In [3]:
pregnant = tf.feature_column.numeric_column('Number_pregnant', dtype=tf.int32)
glucose = tf.feature_column.numeric_column('Glucose_concentration')
blood = tf.feature_column.numeric_column('Blood_pressure')
triceps = tf.feature_column.numeric_column('Triceps')
insulin = tf.feature_column.numeric_column('Insulin')
bmi = tf.feature_column.numeric_column('BMI')
pedigree = tf.feature_column.numeric_column('Pedigree')
age = tf.feature_column.numeric_column('Age')

In [14]:
base_columns = [
    pregnant, 
    glucose, 
    blood, 
    triceps, 
    insulin, 
    bmi, 
    pedigree, 
    age]

crossed_columns = [
    tf.feature_column.crossed_column(['Blood_pressure', 'Age'], hash_bucket_size = 1000)
]

model = tf.estimator.LinearClassifier(feature_columns=base_columns + crossed_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/j5/j768nzj554jbgl0ypv8vxk640000gp/T/tmp07aqmieh', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1233f5240>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [15]:
train_input_func = tf.estimator.inputs.pandas_input_fn(
    x_train,
    y_train,
    batch_size=120,
    num_epochs= None,
    shuffle = False,
)

In [16]:
model.train(input_fn = train_input_func, steps = 1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/j5/j768nzj554jbgl0ypv8vxk640000gp/T/tmp07aqmieh/model.ckpt.
INFO:tensorflow:loss = 83.1777, step = 1
INFO:tensorflow:global_step/sec: 252.577
INFO:tensorflow:loss = 68.833565, step = 101 (0.399 sec)
INFO:tensorflow:global_step/sec: 315.599
INFO:tensorflow:loss = 59.72164, step = 201 (0.317 sec)
INFO:tensorflow:global_step/sec: 280.286
INFO:tensorflow:loss = 56.11753, step = 301 (0.357 sec)
INFO:tensorflow:global_step/sec: 324.86
INFO:tensorflow:loss = 61.20976, step = 401 (0.307 sec)
INFO:tensorflow:global_step/sec: 322.221
INFO:tensorflow:loss = 59.85339, step = 501 (0.310 sec)
INFO:tensorflow:global_step/sec: 272.416
INFO:tensorflow:loss = 49.597347, step = 601 (0.366 sec)
INFO:tensorflow:global_step/sec: 301.034
INFO:tensorflow:loss = 51.256905, step = 701 (0.334 sec)
INFO:tensorflow:global_step/sec: 247.959
INFO:tensorflow:loss = 62.63417, step = 801 (0.405 sec)
INFO:tensorflow:gl

In [17]:
test_input_func = tf.estimator.inputs.pandas_input_fn(
    x_test,
    y_test,
    batch_size=80,
    num_epochs=1000,
    shuffle=True)

In [18]:
test_metrics = model.evaluate(input_fn = test_input_func,steps=1000)

INFO:tensorflow:Starting evaluation at 2018-03-14-04:41:57
INFO:tensorflow:Restoring parameters from /var/folders/j5/j768nzj554jbgl0ypv8vxk640000gp/T/tmp07aqmieh/model.ckpt-1000
INFO:tensorflow:Evaluation [100/1000]
INFO:tensorflow:Evaluation [200/1000]
INFO:tensorflow:Evaluation [300/1000]
INFO:tensorflow:Evaluation [400/1000]
INFO:tensorflow:Evaluation [500/1000]
INFO:tensorflow:Evaluation [600/1000]
INFO:tensorflow:Evaluation [700/1000]
INFO:tensorflow:Evaluation [800/1000]
INFO:tensorflow:Evaluation [900/1000]
INFO:tensorflow:Evaluation [1000/1000]
INFO:tensorflow:Finished evaluation at 2018-03-14-04:42:01
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.7566, accuracy_baseline = 0.6566125, auc = 0.7805904, auc_precision_recall = 0.68070996, average_loss = 0.52646905, global_step = 1000, label/mean = 0.3433875, loss = 42.117523, prediction/mean = 0.34040928


In [19]:
print("test metrics: {}".format(test_metrics))

test metrics: {'accuracy': 0.7566, 'accuracy_baseline': 0.6566125, 'auc': 0.7805904, 'auc_precision_recall': 0.68070996, 'average_loss': 0.52646905, 'label/mean': 0.3433875, 'loss': 42.117523, 'prediction/mean': 0.34040928, 'global_step': 1000}
